<h2>Опечатки и лематизация<h2>

<h4>Импортируем библиотеки<h4>

In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.metrics.distance import jaccard_distance
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.util import ngrams
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('words')
from collections import Counter
from model_selction import model_selection_word_count, model_selection_word_exist, model_selection_tfidf


[nltk_data] Downloading package words to /home/val/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/val/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/val/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
%ls data

p00_tweets.zip*         processedNeutral.csv*
processedNegative.csv*  processedPositive.csv*


<h4>В качестве примера рассмотрим содержимое файла 'processedNegative.csv' после применения метода<h4>

In [16]:
neg_df = pd.read_csv('data/processedNegative.csv').T.reset_index()
correct_words = words.words()
wordnet = WordNetLemmatizer()
neg_token = list()
cor_text = list()
lem_text = list()
uncor_text = list()
for sen in neg_df.values.tolist():
    text = word_tokenize(sen[0])
    cor_text.clear()
    uncor_text.clear()
    for i in range(len(text)):
        word = text[i]
        if word in ["'m", "'ll", "'ve", "'d", "'s"]:
            cor_text.append(word)
        elif len(word) > 1:
            temp = [(jaccard_distance(set(ngrams(word, 2)),
                                    set(ngrams(w, 2))),w)
                    for w in correct_words if w[0]==word[0]]
            try:
                cor_text.append(sorted(temp, key = lambda val:val[0])[0][1])
            except IndexError:
                uncor_text.append(word)
        elif word in ['I', 'a', 'A']:
            cor_text.append(word)
        else:
            uncor_text.append(word)
    lem_text.clear()
    for token, tag in pos_tag(cor_text):
        pos = tag[0].lower()
        if pos not in ['a', 'r', 'n', 'v']:
            pos = 'n'
        lem_text.append(wordnet.lemmatize(token, pos))
    lem_text = [word for word in lem_text if not word in stopwords.words('english')]
    neg_token.extend(lem_text)
    neg_token.extend(uncor_text)
neg_token

['Ho',
 'unhappy',
 'dog',
 'like',
 'though',
 'talk',
 'driver',
 'I',
 "'m",
 'go',
 'say',
 "'d",
 'love',
 'go',
 'Newar',
 'Yorker',
 'since',
 'Triumph',
 "'s",
 'probably',
 'Doeg',
 'anybody',
 'know',
 'Rand',
 "'s",
 'likely',
 'fall',
 'dollar',
 'I',
 'get',
 'money',
 'I',
 'need',
 'change',
 'keep',
 'get',
 'strong',
 'unhappy',
 '?',
 'R',
 'I',
 'miss',
 'go',
 'gig',
 'Liverpudlian',
 'unhappy',
 'Theresa',
 'new',
 'Ricciales',
 'tonight',
 'unhappy',
 '?',
 "'s",
 'A',
 'dye',
 'guy',
 'pop',
 'Asian',
 'translator',
 "'ll",
 'prob',
 'go',
 'around',
 'Aus',
 'unhappy',
 '*',
 'Whig',
 "'s",
 'chair',
 'sit',
 'Isis',
 'I',
 'find',
 'Everyman',
 'know',
 'Yomud',
 "'ve",
 'sham',
 'pu',
 "'re",
 '?',
 '.',
 '.',
 'n',
 'like',
 'jittery',
 'caffeine',
 'make',
 'sad',
 'Mya',
 'area',
 "'s",
 'list',
 'unhappy',
 'think',
 'I',
 "'ll",
 'go',
 'Libby',
 'anyway',
 'I',
 'want',
 'fun',
 'plan',
 'weekend',
 'unhappy',
 'Wend',
 'notice',
 'unhappy',
 '.',
 '?',


<h4>Функция, которая создасть набор данных для обучения моделей<h4>

In [20]:
def lem_misspel_file_to_list(file_name):
    df = pd.read_csv(file_name).T.reset_index()
    correct_words = words.words()
    wordnet = WordNetLemmatizer()
    tokens = list()
    cor_text = list()
    lem_text = list()
    uncor_text = list()
    for sen in df.values.tolist():
        text = word_tokenize(sen[0])
        cor_text.clear()
        uncor_text.clear()
        for i in range(len(text)):
            word = text[i]
            if word in ["'m", "'ll", "'ve", "'d", "'s"]:
                cor_text.append(word)
            elif len(word) > 1:
                temp = [(jaccard_distance(set(ngrams(word, 2)),
                                        set(ngrams(w, 2))),w)
                        for w in correct_words if w[0]==word[0]]
                try:
                    cor_text.append(sorted(temp, key = lambda val:val[0])[0][1])
                except IndexError:
                    uncor_text.append(word)
            elif word in ['I', 'a', 'A']:
                cor_text.append(word)
            else:
                uncor_text.append(word)
        lem_text.clear()
        for token, tag in pos_tag(cor_text):
            pos = tag[0].lower()
            if pos not in ['a', 'r', 'n', 'v']:
                pos = 'n'
            lem_text.append(wordnet.lemmatize(token, pos))
        lem_text = [word for word in lem_text if not word in stopwords.words('english')]
        tokens.extend(lem_text)
        tokens.extend(uncor_text)
    return tokens

def lem_misspell_file_to_df(file_name):
    neg_fn, neut_fn, pos_fn = file_name

    neg_words = Counter(lem_misspel_file_to_list(neg_fn))
    neut_words = Counter(lem_misspel_file_to_list(neut_fn))
    pos_words = Counter(lem_misspel_file_to_list(pos_fn))
    
    unic_words = list(set(neg_words.keys()) | set(neut_words.keys()) | set(pos_words.keys()))

    neg_exist_index = 0
    neut_exist_index = 1
    pos_exist_index = 2
    neg_count_index = 3
    neut_count_index = 4
    pos_count_index = 5
    word_count_index = 6
    neg_tfidf_index = 7
    neut_tfidf_index = 8
    pos_tfidf_index = 9

    df = np.zeros((len(unic_words), 10))
    for i, word in enumerate(unic_words):
        if word in neg_words.keys():
            df[i,neg_exist_index] = 1
            df[i,neg_count_index] = neg_words[word]
        if word in neut_words.keys():
            df[i,neut_exist_index] = 1
            df[i,neut_count_index] = neut_words[word]
        if word in pos_words.keys():
            df[i,pos_exist_index] = 1
            df[i,pos_count_index] = pos_words[word]

    df[:,word_count_index] = df[:,neg_count_index] + df[:,neut_count_index] + df[:,pos_count_index]
    df[:,neg_tfidf_index] = df[:,neg_count_index] / df[:,word_count_index]
    df[:,neut_tfidf_index] = df[:,neut_count_index] / df[:,word_count_index]
    df[:,pos_tfidf_index] = df[:,pos_count_index] / df[:,word_count_index]

    spell_df = pd.DataFrame(df, columns=[
        'Negative', 'Neutral', 'Positive',
        'Negative counts', 'Neutral counts', 'Positive counts', 'Word counts',
        'Negative TFIDF', 'Neutral TFIDF', 'Positive TFIDF'])
    spell_df["word"] = unic_words
    return spell_df

<h4>Узнаем, как называются остальные файлы, содержащие исходный набор данных<h4>

In [4]:
%ls data

p00_tweets.zip*         processedNeutral.csv*
processedNegative.csv*  processedPositive.csv*


<h4>Создадим набор данных для обучения<h4>

In [21]:
file_names = ('data/processedNegative.csv', 'data/processedNeutral.csv', 'data/processedPositive.csv')
lem_misspel_df = lem_misspell_file_to_df(file_names)
lem_misspel_df

,Negative,Neutral,Positive,Negative counts,Neutral counts,Positive counts,Word counts,Negative TFIDF,Neutral TFIDF,Positive TFIDF,word
0,1.0,0.0,1.0,3.0,0.0,2.0,5.0,0.600000,0.000000,0.400000,sweet
1,1.0,1.0,1.0,1.0,1.0,3.0,5.0,0.200000,0.200000,0.600000,Vern
2,1.0,0.0,1.0,1.0,0.0,1.0,2.0,0.500000,0.000000,0.500000,Celeste
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.000000,1.000000,0.000000,affordable
4,1.0,1.0,1.0,1.0,1.0,7.0,9.0,0.111111,0.111111,0.777778,application
...,...,...,...,...,...,...,...,...,...,...,...
5621,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.000000,1.000000,0.000000,walk
5622,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.000000,0.000000,1.000000,Pondo
5623,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.000000,1.000000,0.000000,hearing
5624,1.0,1.0,0.0,1.0,1.0,0.0,2.0,0.500000,0.500000,0.000000,Ricciales


<h4>Сохраним полученный набор данных, чтобы уменьшить ожидание при демонтсрации<h4>

In [22]:
lem_misspel_df.to_csv('data/lem_misspel.csv')

In [23]:
lem_misspel_df = pd.read_csv('data/lem_misspel.csv')
unic_words = lem_misspel_df.word.tolist()

In [24]:
unic_words

['sweet',
 'Vern',
 'Celeste',
 'affordable',
 'application',
 'elder',
 'sang',
 'Hindu',
 'guitar',
 'Nigel',
 'recommendee',
 'criticism',
 'Australia',
 'cadet',
 'notice',
 'Secale',
 'cheapery',
 'Jayant',
 'Lot',
 'hi',
 'calf',
 'Eremian',
 'monsterhood',
 'December',
 'deserve',
 'already',
 'Mike',
 'research',
 'Decian',
 'frozen',
 'seal',
 'contract',
 'unhappiness',
 '}',
 'cast',
 'win',
 'perfectly',
 'fear',
 '264',
 'next',
 '.a',
 'ply',
 'Sedan',
 'wait',
 'hot',
 'everybody',
 'yurt',
 'lightly',
 'mess',
 'Uncompahgre',
 'final',
 'rejecter',
 'Bud',
 'Clark',
 'vehicle',
 'attach',
 'Kids',
 'without',
 'entitle',
 'becap',
 'hoop',
 'prefer',
 'Regga',
 'bedroom',
 'loud',
 'summoner',
 'Aptera',
 'carse',
 'session',
 'Phlomis',
 '908sq',
 'Kurd',
 'map',
 'footpath',
 'reactor',
 'team',
 'Dada',
 'Magellan',
 'talk',
 'purchase',
 'cow',
 'tete',
 'reading',
 'Dehaites',
 'surround',
 'widen',
 'leer',
 'Savara',
 'frustration',
 'Anacharis',
 'Gujrati',
 'sh

<h4>Узнаем полученную точность модели<h4>

In [25]:
word_exist_accuracy_score = model_selection_word_exist(lem_misspel_df, unic_words)
word_count_accuracy_score = model_selection_word_count(lem_misspel_df, unic_words)
tfidf_accuracy_score = model_selection_tfidf(lem_misspel_df, unic_words)

/home/val/.local/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [26]:
print(f"""Accuracy score by word exist: {word_exist_accuracy_score}
Accuracy score by word count: {word_count_accuracy_score}
Fccuracy score by tfidf: {tfidf_accuracy_score}""")

Accuracy score by word exist: 0.5133214920071048
Accuracy score by word count: 0.933392539964476
Fccuracy score by tfidf: 0.522202486678508
